In [ ]:
import pandas as pd
import numpy as np
import re
import gensim
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
from wordcloud import WordCloud
from gensim import corpora, models, similarities
import pyLDAvis
from pprint import pprint
import pickle 
import os
import matplotlib.pyplot as plt
from gensim.models import CoherenceModel
from gensim.test.utils import datapath
import random
from gensim.models.ldamodel import LdaModel
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from gensim.corpora import MmCorpus
import mymodule

In [ ]:
data = pd.read_csv('dataset/DatasetLegal.csv')
str_answer = data['answer'].astype(str)
str_answer = str_answer.map(lambda x: re.sub('[,.!?#/]', '', x))
str_question = data['question'].astype(str)
str_question = str_question.map(lambda x: re.sub('[,.!?*/]', '', x))

In [ ]:
sentense_token = []
for i in range(len(str_answer)):
  sentense_token.append(str_question[i])
  sentense_token.append(str_answer[i])

train_data = sentense_token[:13062]
test_data = sentense_token[13062:]

In [ ]:
word_token_answer = []
for sentense in train_data:
  word = word_tokenize(sentense, engine='newmm')
  word_token_answer.append(word)

In [ ]:
stopwords = list(thai_stopwords())
read_stopwords = pd.read_csv('dataset/add_stopwords.csv')
add_stopwords = read_stopwords['stopword'].values.tolist()
processed_answer = []
for sentense in word_token_answer:
  each_sentense = []
  for word in sentense:
    if(word not in stopwords + add_stopwords):
      each_sentense.append(word)
  processed_answer.append(each_sentense)
print(processed_answer[0][:8])

In [ ]:
id2word = corpora.Dictionary(processed_answer)
# print(id2word)
with open('model/id2word.pkl', 'wb') as f:
    pickle.dump(id2word, f)

In [ ]:
corpus = []
for text in processed_answer:
  vec = id2word.doc2bow(text)
  corpus.append(vec)

In [ ]:
num_topics = 6
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       iterations=100,
                                       chunksize=2000,
                                       passes=10,
                                       alpha=0.9,
                                       eta=0.5
                                       )

In [ ]:
with open('model/lda_model.pkl', 'wb') as f:
    pickle.dump(lda_model, f)

In [ ]:
lda_model.show_topics(num_topics=6, num_words=10)

In [ ]:
import pickle
topic_dict = {
    0 : "succession",
    1 : "violation",
    2 : "family",
    3 : "criminal",
    4 : "contract",
    5 : "labor",
}

with open('model/topic_dict.pkl', 'wb') as f:
    pickle.dump(topic_dict, f)


In [ ]:
# import pyLDAvis.gensim_models as gensimvis
# num_topics = 6
# with open('model/id2word.pkl', 'rb') as f:
#   id2word = pickle.load(f)
# with open('model/lda_model.pkl', 'rb') as f:
#   lda_model = pickle.load(f)
# pyLDAvis.enable_notebook()
# LDAvis_data_filepath = os.path.join('./ldavis_prepared_'+str(num_topics))

# LDAvis_data_filepath
# if 1 == 1:
#     LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
#     with open(LDAvis_data_filepath, 'wb') as f:
#         pickle.dump(LDAvis_prepared, f)
# with open(LDAvis_data_filepath, 'rb') as f:
#     LDAvis_prepared = pickle.load(f)
# pyLDAvis.save_html(LDAvis_prepared, './ldavis_prepared_'+ str(num_topics) +'.html')
# LDAvis_prepared

The sections below working on lda model in question part

In [ ]:
sentense_token_question = []
# for each_question in str_question:
#   sentense_token_question.append(each_question)
for i in range(6531):
  sentense_token_question.append(str_question[i])
# print(len(question_data))
topic_question = []
corpus_question = []
for sentense in sentense_token_question:
  processed_question = mymodule.preprocess(sentense)
  corpus_question.append(processed_question)
  topic = lda_model.get_document_topics(processed_question)
  topic_question.append(topic)

In [ ]:
with open('model/question_lda.pkl', 'wb') as f:
    pickle.dump(topic_question, f)
MmCorpus.serialize('model/corpus_question.mm', corpus_question)

The sections below are in the process of being updated with new documents.

In [ ]:
new_doc = random.choice(test_data)
print(new_doc)
test_doc = mymodule.preprocess(new_doc)

In [ ]:
with open('model/question_lda.pkl', 'rb') as f:
    topic_question = pickle.load(f)
with open('model/topic_dict.pkl', 'rb') as f:
    topic_dict = pickle.load(f)
new_doc_topics = lda_model.get_document_topics(test_doc)
new_doc_topics_dict = {topic_dict[topic]: prob for topic, prob in new_doc_topics}
print(new_doc_topics_dict)

In [ ]:
# print(index)
# print(corpus_lda)
# print(sims)

In [ ]:
corpus = corpus_question
data = pd.read_csv('dataset/DatasetLegal.csv')
with open('model/id2word.pkl', 'rb') as f:
  id2word = pickle.load(f)
corpus_lda = lda_model[corpus]
index = similarities.MatrixSimilarity(corpus_lda, num_features=len(id2word))
sims = index[new_doc_topics]
sims_sorted = sorted(enumerate(sims), key=lambda item: -item[1])
print(f"Topic distribution for new document : {new_doc_topics}\n{new_doc}\n")
for doc_id, similarity in sims_sorted[:5]:
  print(f"Document ID: {doc_id}, Similarity score: {similarity}")
  print(data.answer[doc_id])
  print("Topic distribution for similar document : ")
  for num, dis in corpus_lda[doc_id]:
    print(f"\t({topic_dict.get(num)}, {'%.5f' %dis})")